# BN assignment failure from LEAR

In [1]:
import os
import pandas as pd
import csv
import matplotlib
from datetime import datetime, timedelta
from IPython.core.display import HTML
from cloud_sql_connector import DBConfig, getconn
import pg8000
import sqlalchemy

%load_ext sql
%config SqlMagic.displaylimit = 5

## Database Connection Setup

This notebook now uses the custom bcgov cloud-sql-connector library to connect to the database in GCP. 

Required environment variables:
- `DB_USER`: Database username
- `DB_NAME`: Database name  
- `DB_INSTANCE_CONNECTION_NAME`: Cloud SQL instance connection name (format: `project:region:instance`)

The connection automatically uses the `business` schema and handles IAM authentication through the custom connector.

This will create the connection to the Cloud SQL database using the custom bcgov cloud-sql-connector and prep the jupyter magic for SQL

In [2]:
# Initialize DBConfig for Cloud SQL connection
config = DBConfig(
    instance_name=os.getenv('DB_INSTANCE_CONNECTION_NAME'),  # Cloud SQL instance connection name
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    ip_type="public",  # or "private" depending on your setup
    schema="business"  # Set the schema directly in config
)

def get_conn():
    """Create a connection to Google Cloud SQL using the custom cloud-sql-connector."""
    return getconn(config)

In [3]:
# Create SQLAlchemy engine using Cloud SQL connector
engine = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=get_conn,
)

print("Cloud SQL engine created successfully!")
print(f"Engine: {engine}")

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text("SELECT 1 as test"))
        print("✅ Database connection test successful!")
        print(f"Test result: {result.fetchone()}")
except Exception as e:
    print(f"❌ Connection failed: {e}")

Cloud SQL engine created successfully!
Engine: Engine(postgresql+pg8000://)
✅ Database connection test successful!
Test result: (1,)
✅ Database connection test successful!
Test result: (1,)


BN Assignment Failure: Audit of completed filings without tax_id

In [4]:
# BN Assignment Failure Query
query = """
SELECT b.identifier, b.founding_date, f.filing_type
FROM business.businesses b
JOIN business.filings f ON b.id = f.business_id 
WHERE 
    b.tax_id IS NULL AND
    b.legal_type IN ('BC', 'BEN', 'CC', 'ULC', 'C', 'CBEN', 'CCC', 'CUL') AND
    f.filing_type IN ('incorporationApplication', 'amalgamationApplication', 'continuationIn')
ORDER BY b.identifier
"""

try:
    bn_failure = pd.read_sql(query, engine)
    print(f"✅ BN failure query successful! Found {len(bn_failure)} rows")
    if not bn_failure.empty:
        display(bn_failure.head())
except Exception as e:
    print(f"❌ BN failure query failed: {e}")
    bn_failure = pd.DataFrame()  # Create empty DataFrame for error handling


✅ BN failure query successful! Found 6 rows


,identifier,founding_date,filing_type
0,BC1395800,2023-01-12 17:36:31.163108+00:00,incorporationApplication
1,BC1419266,2023-05-31 03:23:26.397110+00:00,incorporationApplication
2,BC1508370,2024-10-24 23:07:54+00:00,incorporationApplication
3,BC1526729,2025-02-13 18:44:22.969556+00:00,incorporationApplication
4,BC1530986,2025-03-12 00:21:29.218084+00:00,incorporationApplication


In [5]:
corp_audit = bn_failure.copy()  # Use the DataFrame from the previous query

datestr = datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d')
filename =  os.path.join(os.getcwd(), r'data/')+'bn_failure_' + datestr +'.csv'

with open(filename, 'w') as f:
    if corp_audit.empty:
        f.write('No Data Retrieved for BN assignment failure')         
    else:  
        f.write('BN assignment failure ' + ':\n')            
        corp_audit.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n') 